- batch size 현재 512 -> 1024로 진행 가능한지
- 아래 코드 잘 실행되면 weekend, price 컬럼 추가해서 돌리기

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# [Hyperparameter Setting]
CFG = {
    'TRAIN_WINDOW_SIZE': 90,  # 90일치로 학습
    'PREDICT_SIZE': 21,  # 21일치 예측
    'EPOCHS': 20,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 512,
    'SEED': 41,
    'NUM_ENCODER_LAYERS': 2,  # Number of Transformer Encoder layers
    'NUM_HEADS': 8,  # Number of attention heads in Transformer
    'HIDDEN_SIZE': 512,  # Hidden size of Transformer model
    'DROPOUT': 0.1,  # Dropout rate in Transformer
}


# [Import data]
train_data = pd.read_csv('data/train.csv').drop(columns=['ID', '제품'])


# Data Scaling
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()


label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    num_rows = len(data)
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])

        window = sales_data[-train_size:]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len * 0.2):]
val_target = train_target[-int(data_len * 0.2):]
train_input = train_input[:-int(data_len * 0.2)]
train_target = train_target[:-int(data_len * 0.2)]

# [Custom Dataset]
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


In [ ]:
# [Transformer Model]
class TransformerModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=CFG['HIDDEN_SIZE'], output_size=CFG['PREDICT_SIZE'],
                 num_encoder_layers=CFG['NUM_ENCODER_LAYERS'], num_heads=CFG['NUM_HEADS'], dropout=CFG['DROPOUT']):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout),
            num_encoder_layers)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.ReLU()

    def forward(self, x):
        x = self.embedding(x)  # Shape: (B, TRAIN_WINDOW_SIZE, HIDDEN_SIZE)
        x = x.permute(1, 0, 2)  # Shape: (TRAIN_WINDOW_SIZE, B, HIDDEN_SIZE)

        # Transformer Encoder
        x = self.transformer_encoder(x)  # Shape: (TRAIN_WINDOW_SIZE, B, HIDDEN_SIZE)
        x = x.permute(1, 0, 2)  # Shape: (B, TRAIN_WINDOW_SIZE, HIDDEN_SIZE)

        # Fully connected layer
        x = self.actv(self.fc(x[:, -1, :]))

        return x

# [Model Training]
def train_transformer(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []

        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation_transformer(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')

    return best_model

def validation_transformer(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())

    return np.mean(val_loss)

# [Run !!]
transformer_model = TransformerModel()
optimizer = torch.optim.Adam(params=transformer_model.parameters(), lr=CFG["LEARNING_RATE"])
trained_transformer_model = train_transformer(transformer_model, optimizer, train_loader, val_loader, device)

In [ ]:
# [Model Inference]
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # Move model output to CPU and convert to numpy array
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

transformer_pred = inference(trained_transformer_model, test_loader, device)

# [Inverse Scaling and Post-processing]
for idx in range(len(transformer_pred)):
    transformer_pred[idx, :] = transformer_pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# Post-processing
transformer_pred = np.round(transformer_pred, 0).astype(int)

# [Submission]
submit = pd.read_csv('data/sample_submission.csv')
submit.iloc[:, 1:] = transformer_pred
submit.to_csv('data/transformer_submit.csv', index=False)